In [24]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier

In [25]:
dataset=pd.read_csv("Social_Network_Ads.csv")
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)
dataset.columns

Index(['User ID', 'Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [26]:
independent=dataset[['Age', 'EstimatedSalary','Gender_Male']]
dependent=dataset[['Purchased']]
X_train,X_test,y_train,y_test=train_test_split(independent,dependent,test_size=1/3,random_state=0)
    

In [27]:
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)


In [29]:
param_grid = {'criterion':['gini','entropy'],
              'max_features': ['auto','sqrt','log2'],
              'n_estimators':[10,100]} 
grid = GridSearchCV(RandomForestClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1') 
   
# fitting the model for grid search 
grid.fit(X_train, y_train) 
y_pred=grid.predict(X_test)
cm=confusion_matrix(y_test,y_pred)
print(cm)
class_report=classification_report(y_test,y_pred)
print(class_report)
roc=roc_auc_score(y_test,y_pred)
print(roc)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[[79  6]
 [ 5 44]]
              precision    recall  f1-score   support

           0       0.94      0.93      0.93        85
           1       0.88      0.90      0.89        49

    accuracy                           0.92       134
   macro avg       0.91      0.91      0.91       134
weighted avg       0.92      0.92      0.92       134

0.9136854741896759


C:\Users\Nishok\miniconda3\envs\ml_env\Lib\site-packages\sklearn\model_selection\_validation.py:516: FitFailedWarning: 
20 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
14 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Nishok\miniconda3\envs\ml_env\Lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Nishok\miniconda3\envs\ml_env\Lib\site-packages\sklearn\base.py", line 1358, in wrapper
    estimator._validate_params()
  File "C:\Users\Nishok\miniconda3\envs\ml_env\Lib\site-packages\sklearn\base.py", line 471, in _validate_params
    validate_parameter_constrai

In [9]:

param_grid = {
    "kernel": ["rbf", "linear"],       # <-- correct key
    "C": [0.5, 1, 3, 10],
    "gamma": ["scale", 0.1, 0.01],     # gamma used for rbf/poly/sigmoid; ignored for linear
}
grid=GridSearchCV(SVC(probability=True),param_grid,refit=True,verbose=3,cv=5,n_jobs=-1,scoring='f1_weighted')
grid.fit(X_train,y_train)
            

Fitting 5 folds for each of 24 candidates, totalling 120 fits


C:\Users\Nishok\miniconda3\envs\ml_env\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,estimator,SVC(probability=True)
,param_grid,"{'C': [0.5, 1, ...], 'gamma': ['scale', 0.1, ...], 'kernel': ['rbf', 'linear']}"
,scoring,'f1_weighted'
,n_jobs,-1
,refit,True
,cv,5
,verbose,3
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,C,1


In [20]:
y_pred=grid.predict(X_test)

In [11]:
cm=confusion_matrix(y_test,y_pred)
print(cm)
class_report=classification_report(y_test,y_pred)
print(class_report)

[[77  8]
 [ 4 45]]
              precision    recall  f1-score   support

           0       0.95      0.91      0.93        85
           1       0.85      0.92      0.88        49

    accuracy                           0.91       134
   macro avg       0.90      0.91      0.91       134
weighted avg       0.91      0.91      0.91       134



In [15]:
roc=roc_auc_score(y_test,y_pred)
print(roc)

0.9121248499399759


In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix

df = pd.read_csv("Social_Network_Ads.csv")

X = df.drop(columns=["User ID", "Purchased"])
y = df["Purchased"]
pre = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown="ignore"), ["Gender"])
], remainder="passthrough")

clf = DecisionTreeClassifier(random_state=42)

pipe = Pipeline([
    ("pre", pre),
    ("clf", clf),
])

# --- Hyperparameter grid (strong, sensible ranges) ---
param_grid = {
    "clf__criterion": ["gini", "entropy"],                # split quality
    "clf__max_depth": [3, 4, 5, 6, 8, 10, None],          # control complexity
    "clf__min_samples_split": [2, 5, 10, 20],             # min samples to split
    "clf__min_samples_leaf": [1, 2, 4, 6, 8],             # min samples in a leaf
    "clf__max_features": [None, "sqrt", "log2", 0.6, 0.8, 1.0],  # features per split
    "clf__splitter": ["best", "random"],                  # "random" can generalize better sometimes
    "clf__class_weight": [None, "balanced"],              # handle mild imbalance
    "clf__min_impurity_decrease": [0.0, 1e-4, 5e-4, 1e-3],
    "clf__ccp_alpha": [0.0, 0.0005, 0.001, 0.005, 0.01],  # post-pruning
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

gs = GridSearchCV(
    pipe, param_grid,
    scoring="f1",           
    cv=cv, n_jobs=-1, verbose=2, refit=True
)

X_tr, X_te, y_tr, y_te = train_test_split(
    X, y, test_size=1/3, stratify=y, random_state=42
)

gs.fit(X_tr, y_tr)
print("Best params:", gs.best_params_)

y_pred = gs.predict(X_te)
print(confusion_matrix(y_te, y_pred))
print(classification_report(y_te, y_pred, digits=2))


Fitting 5 folds for each of 134400 candidates, totalling 672000 fits
Best params: {'clf__ccp_alpha': 0.0, 'clf__class_weight': None, 'clf__criterion': 'gini', 'clf__max_depth': 3, 'clf__max_features': 'sqrt', 'clf__min_impurity_decrease': 0.0, 'clf__min_samples_leaf': 1, 'clf__min_samples_split': 10, 'clf__splitter': 'best'}
[[78  8]
 [ 4 44]]
              precision    recall  f1-score   support

           0       0.95      0.91      0.93        86
           1       0.85      0.92      0.88        48

    accuracy                           0.91       134
   macro avg       0.90      0.91      0.90       134
weighted avg       0.91      0.91      0.91       134

